In [1]:
# Notebook developed around BigFix dataset to understand Database software at the agency
# it is backed by a postgresql database. See README.md on building the database.

import sys
sys.path.append("../bin")
from utility import *
from info import *

from IPython.display import display, HTML

In [2]:
print (RELATIONAL_DB_LIST)

['4th Dimension', 'Adabas D', 'Alpha Five', 'Apache Derby', 'Aster Data', 'Amazon Aurora', 'Altibase', 'CA Datacom', 'CA IDMS', 'Clarion', 'ClickHouse', 'Clustrix', 'CSQL', 'CUBRID', 'DataEase', 'Database Management Library', 'Dataphor', 'dBase', 'Derby aka Java DB', 'Empress Embedded Database', 'EXASolution', 'EnterpriseDB', 'eXtremeDB', 'FileMaker Pro', 'Firebird', 'FrontBase', 'Google Fusion Tables', 'Greenplum', 'GroveSite', 'H2', 'Helix database', 'HSQLDB', 'IBM DB2', 'IBM Lotus Approach', 'IBM DB2 Express-C', 'Infobright', 'Informix', 'Ingres', 'InterBase', 'InterSystems Caché', 'LibreOffice Base', 'Linter', 'MariaDB', 'MaxDB', 'MemSQL', 'Microsoft Access', 'Microsoft Jet Database Engine', 'Microsoft SQL Server', 'Microsoft SQL Server Express', 'SQL Azure', 'Cloud SQL Server', 'Microsoft Visual FoxPro', 'Mimer SQL', 'MonetDB', 'mSQL', 'MySQL', 'Netezza', 'NexusDB', 'NonStop SQL', 'NuoDB', 'Omnis Studio', 'Openbase', 'OpenLink Virtuoso', 'OpenLink Virtuoso Universal Server', 'Open

In [3]:
# make the connection to the database
CONN = create_connection('apps2')

In [11]:
# list of terms we DONT want to have in the name/path
DB_CONSTRAINT_TERMS = ['install', 'uninstall', 'ODBC', 'JDBC', 'updater', 'document', 'app']

def db_filter_constraint (db_software):
    db_constraint = f"""where install_location ilike '%{db_software}%'"""
    for term in DB_CONSTRAINT_TERMS:
        db_constraint += f""" and install_location not ilike '%{term}%'"""
    return db_constraint

#print (db_filter_constraint())

# Define a software search method or 2 for us
def db_software (software, conn=CONN):
    ''' Simple program to automate software searches '''
    
    query = Query (conn, 'software_map', db_filter_constraint(software))
    num_results = len(query.result)
    
    if num_results > 0:
        display(HTML(query.result.to_html()))
        
def db_software_histo (software, conn=CONN):
    '''Program to provide breakdown of types of software '''

    query = HistoQuery(conn, 'software_map', 'software_name', db_filter_constraint(software))
    num_results = len(query.result)
    
    if num_results > 0:
        display(HTML(query.result.to_html()))
        
def db_software_count (software, table='software_map', conn=CONN):
    '''Determine count of matching software'''
    
    query = CountQuery (conn, table, db_filter_constraint(software))
    num_results = len(query.result)
    
    return pd.DataFrame.from_dict({'software' : [software], 'num' : [num_results]})

def count_rdbs_types():
    '''Count out all of the db software we can find'''
    
    data = []
    for db_software in RELATIONAL_DB_LIST:
        data.append(db_software_count(db_software))
    
    display(HTML(pd.concat(data).sort_values(['num'], ascending=False).to_html()))
    
def count_rdbs_installs():
    
    data = []
    for db_software in RELATIONAL_DB_LIST:
        data.append(db_software_count(db_software, table='device_software'))
        
    display(HTML(pd.concat(data).sort_values(['num'], ascending=False).to_html()))

In [12]:
# count_rdbs_types()

In [13]:
count_rdbs_installs()

,num,software
0,5864,Microsoft SQL Server
0,1084,MySQL
0,475,FileMaker Pro
0,412,Oracle
0,90,Firebird
0,86,PostgreSQL
0,34,SQL Anywhere
0,19,H2
0,12,Microsoft Visual FoxPro
0,10,MariaDB


In [8]:
# for a more indepth analysis of particular DB software lets
# get a histogram breaking down all of the types we match
db_software_histo ('microsoft sql server')

,software_name,num
0,Microsoft SQL Server Setup Support Files (Engl...,14
1,Microsoft SQL Server Native Client,9
2,Microsoft SQL Server VSS Writer,8
3,Microsoft SQL Server Management Objects Collec...,6
4,Microsoft SQL Server Compact 3.5 SP2 ENU,6
5,Microsoft SQL Server Compact 3.5 SP2 x64 ENU,5
6,Microsoft SQL Server Compact 3.5 SP1 English,4
7,Microsoft SQL Server 2005 Backward compatibility,4
8,Microsoft SQL Server Compact 3.5 ENU,3
9,Microsoft SQL Server Compact 3.5 for Devices ENU,3
